In [17]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from os import walk
import re
import glob
from xgboost import XGBClassifier
from sklearn import preprocessing

In [18]:
l = []
for name in glob.glob('/kaggle/input/im-somewhat-of-a-cybersecurity-analyst-myself/training/uploads/*.csv'):
    temp = pd.read_csv(name, low_memory=False, nrows = 20000)
    l.append(temp)
df = pd.concat(l, axis =0)

In [19]:
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing[percent_missing>30]

default_browser     95.1580
org_id              30.8744
pua_mode            99.9767
smart_screen        35.5280
processor_class     99.5841
battery_type        71.0012
flighting_status    83.0637
threshold_choice    63.4780
wim_boot_status     63.3868
dtype: float64

In [20]:
df.drop(['default_browser', 'org_id', 'pua_mode', 'smart_screen', 'processor_class', 'battery_type', 'flighting_status', 'threshold_choice', 'wim_boot_status'], axis = 1, inplace = True)

In [21]:
df_ = df.select_dtypes(['int16', 'int32', 'int64', 'float16', 'float32'])
df_.head()

,machine_id,is_beta,is_sxs_passive,has_tpm,country_id,local_name_id,os_build,os_suite,auto_sample_opt_in,has_optical_drive,build_number,build_revision,ui_locale_identifier,os_type_status,secure_boot_status,touch_support,pen_support,malware_status
0,400000,0,0,1,91,75,17134,256,0,0,17134,165,31,0,0,0,0,0
1,400001,0,0,1,160,170,9600,768,0,0,10586,494,115,0,1,0,0,0
2,400002,0,0,1,97,124,17134,768,0,0,17134,167,72,0,1,1,1,1
3,400003,0,0,1,120,75,17134,256,0,1,17134,191,31,0,0,0,0,1
4,400004,0,0,1,220,72,16299,768,0,0,16299,371,31,0,1,0,0,0


In [23]:
column = np.array(list(df_.columns))
for i in range(0,len(df_.columns)):
        x=df_[column[i]].mean()
        df_[column[i]].fillna('x', inplace = True)
df_.drop_duplicates(inplace = True)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [24]:
Z = pd.read_csv('/kaggle/input/im-somewhat-of-a-cybersecurity-analyst-myself/test.csv/test.csv', low_memory = False)
percent_missing_ = Z.isnull().sum() * 100 / len(Z)
percent_missing_[percent_missing_>30]

default_browser     95.141659
org_id              30.797481
pua_mode            99.974117
smart_screen        35.594264
processor_class     99.586916
battery_type        71.038253
flighting_status    83.026421
threshold_choice    63.511534
wim_boot_status     63.426056
dtype: float64

In [25]:
Z.drop(['default_browser', 'org_id', 'pua_mode', 'smart_screen', 'processor_class', 'battery_type', 'flighting_status', 'threshold_choice', 'wim_boot_status'], axis = 1, inplace = True)

In [26]:
Z_ = Z.select_dtypes(['int16', 'int32', 'int64', 'float16', 'float32'])

In [33]:
frame_ = df_.drop('malware_status', axis =1)
X = frame_.iloc[:, 0:17]
Y = df['malware_status']

In [27]:
seed = 42
test_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [28]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [29]:
y_predicted = model.predict(X_test)
predictions = [round(value) for value in y_predicted]

In [30]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print("accuracy: %.2f%%" %(accuracy))

accuracy: 0.56%


In [31]:
z_predicted = model.predict(Z_)
predictions = [round(value) for value in z_predicted]

In [32]:
pd.DataFrame({
    'machine_id': Z_['machine_id'],
    'malware_status': z_predicted
}).to_csv('submission.csv', index=False)